# Create an urban & water mask

Using NVISv6 to define locations of inland water bodies. Urban areas are from ozwald on the NCI.

In [ ]:
import os
import odc.geo
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
from datacube.utils.dask import start_local_dask
client = start_local_dask(mem_safety_margin='2Gb')
client

## Make a 5 km urban and water mask

In [ ]:
urban = xr.open_dataarray('/g/data/os22/chad_tmp/AusEFlux/data/urban_mask_5km.nc').rename({'y':'latitude','x':'longitude'})

water = xr.open_dataarray('/g/data/os22/chad_tmp/Aus_phenology/data/NVISv6_5km.nc') ##24=inland water
water = xr.where(water==24,1,0)

mask = (urban | water)

mask.rename({'latitude':'y','longitude':'x'}).astype(np.int16).to_netcdf('/g/data/os22/chad_tmp/AusEFlux/data/urban_water_mask_5km.nc')

In [ ]:
mask.plot();
plt.title('Urban and water mask 5km');

## Make a 1 km urban and water mask

In [ ]:
urban = xr.open_dataarray('/g/data/os22/chad_tmp/AusEFlux/data/urban_mask_1km.nc')

In [ ]:
ds = rxr.open_rasterio('/g/data/os22/chad_tmp/climate-carbon-interactions/data/aus6_0e_mvg.tif', chunks=(dict(x=250,y=250))).squeeze().drop_vars('band')
ds = ds.where(ds<99).astype('float32')
ds = assign_crs(ds, crs='EPSG:3577')

### Reproject

In [ ]:
#reproject
ds = ds.odc.reproject(how=urban.odc.geobox, resampling='mode')
ds = ds.rename('NVISv6_5km')
ds = round_coords(ds)
ds = ds.where(ds<99)
ds = ds.compute()

In [ ]:
ds = ds.rename({'latitude':'y','longitude':'x'})

In [ ]:
water = xr.where(ds==24,1,0)

mask = (urban | water)

mask.astype(np.int16).to_netcdf('/g/data/os22/chad_tmp/AusEFlux/data/urban_water_mask_1km.nc')

In [ ]:
mask.plot();
plt.title('Urban and water mask 1km');

## Mask the predictions

This is necessary because I had failed to mask water during the production run. Will be redundant on any future run because the urban and water mask is now in the production code.

In [ ]:
base = f'/g/data/os22/chad_tmp/AusEFlux/'

# ----NEE-------------------------------
folder = base+f'results/AusEFlux/NEE/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
nee = xr.open_mfdataset(files)
nee = assign_crs(nee, crs='EPSG:4326')
nee.attrs['nodata'] = np.nan

# ----GPP-------------------------------
folder = base+f'results/AusEFlux/GPP/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
gpp = xr.open_mfdataset(files)
gpp = assign_crs(gpp, crs='EPSG:4326')
gpp.attrs['nodata'] = np.nan


# ----ER-------------------------------
folder = base+f'results/AusEFlux/ER/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
er = xr.open_mfdataset(files)
er = assign_crs(er, crs='EPSG:4326')
er.attrs['nodata'] = np.nan

# ----ER-------------------------------
folder = base+f'results/AusEFlux/ET/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
et = xr.open_mfdataset(files)
et = assign_crs(et, crs='EPSG:4326')
et.attrs['nodata'] = np.nan

### Mask

In [ ]:
mask = mask.rename({'y':'latitude','x':'longitude'})

nee = nee.where(mask!=1).astype(np.float32)
gpp = gpp.where(mask!=1).astype(np.float32)
er = er.where(mask!=1).astype(np.float32)
et = et.where(mask!=1).astype(np.float32)

### export

In [ ]:
 #list of years and export
years = [str(i) for i in range(2003, 2024)]
fluxes = ['GPP', 'ER', 'NEE', 'ET'] 
dss = [gpp,er,nee,et] 

for flux,ds in zip(fluxes,dss):
    for year in years:
        xx = ds.sel(time=year)
        print(flux, year)
        xx.to_netcdf(f'/g/data/os22/chad_tmp/AusEFlux/results/AusEFlux/{flux}_new/AusEFlux_{flux}_1km_quantiles_{year}_v1.2.nc')